In [1]:
from scenarios import generate_cost_projections, generate_scenario_attitudes, MODES_2020, FAST_TRANSITION_MODES_AND_YEARS

generate_scenario_attitudes(MODES_2020, FAST_TRANSITION_MODES_AND_YEARS).tail()

c:\src\canadopt\abetam\scenarios\__init__.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scenario_df = pd.concat([mode_df, empty_df]).sort_index().interpolate()


,Electric furnace,Gas furnace,Heat pump,Oil furnace,Wood or wood pellets furnace
2046,0.2,0.05,0.95,0.05,0.2
2047,0.2,0.05,0.95,0.05,0.2
2048,0.2,0.05,0.95,0.05,0.2
2049,0.2,0.05,0.95,0.05,0.2
2050,0.2,0.05,0.95,0.05,0.2


In [2]:
import pandas as pd
costs = generate_cost_projections(learning_rate=11.1, write_csv=True)
costs


c:\src\canadopt\abetam\scenarios\__init__.py:112: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  costs = pd.concat([empty_frame, costs]).sort_index(level=(1, 0)).interpolate()


,,Electric furnace,Gas furnace,Heat pump,Oil furnace,Wood or wood pellets furnace
year,variable,,,,,
2000,annuity_factor,0.086,0.101,0.11,0.100,0.098
2001,annuity_factor,0.086,0.100,0.11,0.099,0.098
2002,annuity_factor,0.086,0.098,0.11,0.098,0.098
2003,annuity_factor,0.086,0.097,0.11,0.097,0.098
2004,annuity_factor,0.086,0.096,0.11,0.095,0.098
...,...,...,...,...,...,...
2045,specific_fuel_emission,0.020,0.200,0.02,0.500,0.150
2046,specific_fuel_emission,0.020,0.200,0.02,0.500,0.150
2047,specific_fuel_emission,0.020,0.200,0.02,0.500,0.150
